In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl

stopLoss = 0.95
rikaku_day = 80
longTerm = 60

def buyAndHold(stock_code, term=14, start="2010-01-01", end=dt.datetime.now()):
    # load data
    df = yf.download(stock_code, start, end, interval='1d')
    # calculate stock price difference between yesterday and today.
    
    df['Highest'+str(longTerm)] = df.iloc[:, 4].rolling(window=longTerm).max()
    assets = [1]
    buy_asset = 1
    buy_position = False
    counter = 1
    percentChange = []
    
    for i in range(1,len(df)):
        Highest = df['Highest'+str(longTerm)][i-1]
        close = df['Adj Close'][i]
        sell_day = pd.to_datetime(df.index[i])
        
        if buy_position == False:
            assets.append(assets[i-1])
        else:
            assets.append(buy_asset*(close / buy_price))
        
        if i == 1:
            buy_position = True
            buy_price = close
            buy_asset = assets[i]
            
        if (counter == df['Adj Close'].count() - 1):
            if buy_position == True:
                buy_position = False
                percent = (close / buy_price - 1) * 100
                percentChange.append(percent)
                
        counter += 1
    print(percentChange)
    gains = 0
    numGains = 0
    losses = 0
    numLosses = 0
    total_return = 1
    for i in percentChange:
        if i > 0:
            gains += i
            numGains += 1
        else:
            losses += i
            numLosses += 1
        total_return = total_return * ((i/100) + 1)
    total_return = round((total_return - 1)*100, 2)
    print(total_return)
    if numGains > 0:
        ave_gain = gains / numGains
        max_return = max(percentChange)
    else:
        ave_gain = 0
        max_return = np.nan # 'unknown'
    
    if numLosses > 0:
        ave_loss = losses / numLosses
        max_loss = min(percentChange)
        risk_reward_retio = - ave_gain / ave_loss
    else:
        ave_loss = 0
        max_loss = np.nan #'unknown'
        risk_reward_retio = np.nan #'inf'
    
    if numGains > 0 or numLosses > 0:
        batting_retio = numGains / (numGains + numLosses)
    else:
        batting_retio = 0
    
    trades = numGains + numLosses
    
    max_asset = 1
    max_drawdown = 0
    for i in range(len(assets)):
        max_asset = max(max_asset, assets[i])
        now_drawdown = 1 - (assets[i]/max_asset)
        if now_drawdown < 0:
            continue
        max_drawdown = max(max_drawdown, now_drawdown)
    
    results = [trades, total_return, ave_gain, ave_loss, max_return, max_loss, 
               risk_reward_retio, batting_retio, 
               assets[-1], max_drawdown]
    
#     if 'unknown' in results or 'inf' in results:
#         return None
#     else:
    return results
    
    
df = pd.read_csv('~/automation-stock/symbols/nasdaq100.csv')
print(df)
results = []
symbols = []
cnt = 0
for symbol in df['symbol']:
    result = buyAndHold(stock_code=symbol)
    if not result is None:
        symbols.append(symbol)
        results.append(result)

# print(len(results))

columns = ['trades', 'Total return', 'Average Gain', 
           'Average Loss', 'Max Return', 
           'Max Loss', 'Gain/Loss Ratio', 
           'Batting Average', 'Last Asset',
          'Max Drawdown']
df = pd.DataFrame(results, columns=columns, index=symbols)
df.to_csv('./csv/many_buyAndHold.csv')

# failed to download
# BRK.B, BF.B, CTL, ETFC, MYL, NBL,